In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!nvidia-smi

Tue Jun  8 12:08:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=60170f653badfa78a52889cb157134fb4bba5342a419b4be6321da29a688eb37
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [3]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

CPU RAM Free: 26.3 GB
GPU 0 ... Mem Free: 16280MB / 16280MB | Utilization   0%


In [4]:
# Do not run this cell after installing tensorflow js which will come after few steps
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [7]:
from sklearn import preprocessing

In [8]:
import keras
from keras.models import model_from_json
from keras.utils import np_utils

Get data from this location(https://drive.google.com/file/d/1LWTR94ndXOWWsuEMyrJpxk8uzK7mfq77/view?usp=sharing)

In [10]:
## Data read
df=pd.read_csv('pre_processed_data.csv')
df.head()

,intents,content
0,activation_function,In this video as Sudhansu sir explains that we...
1,activation_function,I have some doubts regarding Activation functi...
2,activation_function,Suppose i am using any activation function and...
3,activation_function,I know in case vanishing gradient weight of in...
4,activation_function,I am a student of DLCVNLP 25th batch and have ...


In [11]:
df.isnull().any()

intents    False
content    False
dtype: bool

In [12]:
x = list(df['content'])
y = list(df['intents'])

In [13]:
le = preprocessing.LabelEncoder()
le.fit(y)
le.classes_

array(['Bert_query', 'Cufflinks_installation', 'Dashboard_access',
       'EDA_code', 'activation_function', 'anaconda_installation',
       'anaconda_navigator', 'anaconda_prompt', 'anaconda_query',
       'assignment_KNN', 'assignment_LinearRegression',
       'assignment_data_visulization', 'assignment_evaluation',
       'assignment_exception_handling', 'assignment_link',
       'assignment_matplotlib', 'assignment_numpy', 'assignment_pandas',
       'assignment_powerbi', 'assignment_python', 'assignment_query',
       'assignment_stats', 'assignment_submission',
       'aws_deployment_error', 'aws_lex_account_query',
       'azure_luis_deployment', 'batch_changed', 'batch_details', 'blog',
       'books_availablity_dashboard', 'bot_timing', 'canvas_details_edit',
       'canvas_link', 'career_transition', 'certificates_enquery',
       'chatbot_issue', 'class_link', 'class_query', 'cmd',
       'cnn_question', 'colab_help', 'community_class',
       'community_class_certificates',

In [14]:
def encode(le,labels):
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def decode(le,one_hot):
    dec = np.argmax(one_hot,axis=1)
    return le.inverse_transform(dec)

In [15]:
x_train = x
y_train = encode(le,y)
print(len(x_train),y_train.shape[0])

12146 12146


In [16]:
x_train = np.array(x_train)
x_train

array(['In this video as Sudhansu sir explains that we are using the SoftMax function for the classification in the outer layer the formula for SoftMax is e^x(i)/Sum(e^x(i). How did we get the value for these classes? Because in the video when sir explained SoftMax activation function using the dice example it was pretty simple but can you please explain this',
       'I have some doubts regarding Activation function, can you please help me?',
       'Suppose i am using any activation function and accuracy of my model is not good. How can I make sure that my model not effected by vanishing gradient and exploding gradient problems',
       ...,
       'Zoom app was not working well aind even attending the class,sir was inaudible even,class couldn&#39;t be followed smoothly',
       'is their any change in settings that need to be done for zoom audio?',
       'Good morning sir.. I&apos;m a student of MLD6th batch.. As mentioned Thursday is doubts clearing session. How can I attend the s

In [17]:
y_train = np.array(y_train)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [18]:
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model,load_model
# import keras.backend as K
from keras import backend as K

In [26]:
# !pip uninstall tensorflow tensorflow_hub tensorflowjs
# !pip install tensorflow==2.0.0a0 tensorflow_hub==0.5.0 tensorflowjs==1.2.6

In [19]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

tf.disable_eager_execution()

In [20]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [21]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default" ,as_dict = True)["default"]

In [22]:
tf.disable_eager_execution()
print(tf.get_default_graph())

In [23]:
y_train.shape[1]##89


89

In [24]:
input_text = Input(shape=(1,) ,dtype=tf.string)
embedding = Lambda(ELMoEmbedding,output_shape=(1024,))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(89, activation='softmax')(dense)
model = Model(inputs =[input_text],outputs =pred)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [25]:
%time
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(x_train,y_train,epochs=25,batch_size = 32)
    model.save_weights('./elmo_intent_model_v3.h5')

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 5.48 µs
Train on 12146 samples
Epoch 1/25
12146/12146 [==============================] - 98s 8ms/sample - loss: 1.6864 - accuracy: 0.6123
Epoch 2/25
12146/12146 [==============================] - 74s 6ms/sample - loss: 0.9917 - accuracy: 0.7395
Epoch 3/25
12146/12146 [==============================] - 74s 6ms/sample - loss: 0.7666 - accuracy: 0.7897
Epoch 4/25
12146/12146 [==============================] - 72s 6ms/sample - loss: 0.6266 - accuracy: 0.8227
Epoch 5/25
12146/12146 [==============================] - 73s 6ms/sample - loss: 0.5167 - accuracy: 0.8505
Epoch 6/25
12146/12146 [==============================] - 75s 6ms/sample - loss: 0.4312 - accuracy: 0.8749
Epoch 7/25
12146/12146 [==============================] - 74s 6ms/sample - loss: 0.3514 - accuracy: 0.9005
Epoch 8/25
12146/12146 [==============================] - 74s 6ms/sample - loss: 0.2913 - accuracy: 0.9213
Epoch 9/25
12146/12146 [==============================] -

In [28]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo_intent_model_v3.h5')
    #predicts = model.predict(x_test,batch_size=32)
    predicts = model.predict(x_train[0:5])

In [29]:
predicts[0]

array([4.32321201e-08, 2.34764341e-09, 8.32144423e-11, 6.58813078e-05,
       9.97361958e-01, 9.11784159e-09, 4.80921587e-11, 1.40735201e-10,
       3.32425532e-09, 4.75478268e-11, 2.70648229e-07, 1.64375547e-06,
       3.08696482e-14, 8.56588116e-08, 1.16879455e-07, 9.21336678e-08,
       1.60702825e-07, 3.14388764e-08, 8.76352590e-09, 4.98886821e-10,
       1.18128241e-07, 1.47044200e-11, 2.20351296e-07, 4.26297470e-10,
       1.19833476e-08, 3.90882860e-07, 7.87578960e-11, 6.54995467e-14,
       6.86553115e-12, 1.17398014e-09, 2.69001088e-09, 1.25849969e-10,
       3.25643928e-10, 5.42404524e-11, 4.44107107e-09, 2.58371733e-06,
       7.47142556e-11, 1.08939413e-10, 7.64662084e-11, 9.23796719e-08,
       2.82831735e-07, 2.90262779e-12, 4.31636671e-11, 3.71295807e-11,
       4.25549217e-14, 8.87447171e-10, 6.38939071e-08, 2.40718356e-09,
       1.48389290e-07, 4.70873384e-12, 7.83786827e-13, 4.38234429e-06,
       2.26609043e-08, 2.68266342e-09, 1.05895225e-07, 8.64538197e-06,
      

In [30]:
test=np.array(['i have doubt regrding CNN','I have some doubts regarding Activation function, can you please help me?'])
test

array(['i have doubt regrding CNN',
       'I have some doubts regarding Activation function, can you please help me?'],
      dtype='<U73')

In [31]:
# y_test = decode(le,y_test)
y_preds = decode(le,predicts)
y_preds


array(['activation_function', 'activation_function',
       'activation_function', 'activation_function',
       'activation_function'], dtype='<U31')

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo_intent_model_v3.h5')
    #predicts = model.predict(x_test,batch_size=32)
    predicts = model.predict(test)

In [ ]:
predicts[0]

array([1.66121225e-07, 2.16493642e-10, 3.75704502e-12, 3.35175748e-04,
       9.45675311e-07, 4.04082847e-08, 1.56072517e-07, 2.81700485e-09,
       4.99663755e-10, 1.07139886e-09, 1.72770297e-06, 9.57400381e-09,
       5.98332994e-10, 4.24371045e-08, 2.03687716e-08, 8.36513721e-08,
       8.54693241e-08, 1.85082993e-09, 1.19531961e-07, 2.53957467e-07,
       1.55859689e-05, 4.99080421e-08, 4.82463403e-10, 3.67322617e-09,
       2.14898122e-07, 1.53926949e-09, 6.53800036e-09, 5.00766006e-09,
       1.52113044e-09, 3.79089482e-10, 3.13198667e-09, 7.96281867e-08,
       3.09624479e-11, 3.38649153e-09, 4.23676916e-09, 3.05845589e-08,
       1.20119266e-08, 4.72162842e-09, 2.06158699e-08, 9.99549925e-01,
       6.74205296e-07, 1.31601369e-10, 9.63021954e-11, 4.15859838e-12,
       1.63239512e-13, 1.60402909e-08, 7.96189947e-10, 1.71406480e-08,
       1.70832465e-07, 2.31125258e-11, 1.88693559e-08, 4.03427720e-07,
       9.28851378e-08, 1.47175022e-10, 6.42160194e-06, 9.93432039e-08,
      

In [ ]:
# y_test = decode(le,y_test) ###'i have doubt regrding CNN','I have some doubts regarding Activation function, can you please help me?'
y_preds = decode(le,predicts)
y_preds

array(['cnn_question', 'activation_function'], dtype='<U31')

In [ ]:
# %tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.65862699500002
GPU (s):
0.03951817699999083
GPU speedup over CPU: 41x
